In [ ]:
using DifferentialEquations, CUDA

In [ ]:
using Fields

# Parameters

In [ ]:
L = 2
N = 2^10

tmax = 1.0

x₀ = (L/2, L/2)

n = 2^7
tsave = range(0.0, stop=tmax, length=n)
frac = Float32(N)/n

rsave_idxs = [Integer( floor(i*frac + (j-1)*frac*N) ) + 1 for i in 1:n, j in 1:n]
;

In [ ]:
h = L/(N-1)

x = [((i-1)h, (j-1)h).-x₀ for j in 1:N, i in 1:N]
r = [sqrt(x^2+y^2) for (x,y) in x]
t = 0.0:h/10:tmax
;

### Total saved values:

In [ ]:
MB =  (n^3 * 8) * 10^-6
print("$MB MB")

# Initial conditions + allocs.

### Clean mem

In [ ]:
sol = nothing
GC.gc()

CUDA.reclaim()

### Setup IC

In [ ]:
φ₀   = sin(r)
∂ₜφ₀ = zero(r)
uₕₒₛₜ = cat(φ₀, ∂ₜφ₀; dims=3)
u₀ = CuArray(uₕₒₛₜ)
∇²ϕ  = CuArray(zero(r))
;

### View IC

In [ ]:
#using Plots

#φ₀_plot   = plot(φ₀,  st=:surface)
#∂ₜφ₀_plot = plot(∂ₜφ₀,st=:surface)

#plot(φ₀_plot, ∂ₜφ₀_plot, layout=(1, 2), size=(1300, 400))


# Problem

In [44]:
using CUDA

function δ!(x::CuDeviceVector{Float64}, ε::Float64, out::CuDeviceVector{Float64})
    index = threadIdx().x
    
    absx = abs(x[index])
    
    @inbounds begin
        out[index] = (absx < ε) * (1 - absx/ε)
    end

    return
end

function ring_δ(x, eps)
    x_d = CuArray(x)
    y_d = similar(x_d)
    
    N = length(x_d)
    
    @cuda threads=N δ!(x_d, eps, y_d)
    
    return Array(y_d)
end

ring_δ (generic function with 1 method)

In [47]:
using Plots, Interact

x = -2:0.01:2

@manipulate for ε ∈ 0.1:0.02:1.2, t ∈ 0.0:0.02:1
    y = ring_δ(x.-t, ε)
    plot(x,y)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["ε"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 56, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable{Any}(28), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/joao/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/joao/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/joao/.julia/packages/InteractBase/Qhvxg/src/../assets/all.js"), Asset("css", nothing, "/home/joao/.julia/packages/InteractBase/Qhvxg/src/../assets/style.css"), Asset("css", nothing, "/home/joao/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x00007f9ef7bfbb70, Task (runnable) @0x00007f9ef7bfbb70), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (4, 94481704943152, 4619804387259671055)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.1\",\"0.12\",\"0.14\",\"0.16\",\"0.18\",\"0.2\",\"0.22\",\"0.24\",\"0.26\",\"0.28\",\"0.3\",\"0.32\",\"0.34\",\"0.36\",\"0.38\",\"0.4\",\"0.

In [ ]:
function signumGordon_2d!(uₜ, u, (N, h, ∇²ϕ), t)
    ϕ  = @views u[:,:,1]
    ϕₜ = @views u[:,:,2]

    compute_∇²_2d!(∇²ϕ, ϕ, h)

    # Eq. system
    # φ ≡ ϕₜ
    # φₜ ≡ ϕₜₜ
    φ  = ϕₜ
    φₜ = ∇²ϕ - sign.(ϕ)

    # Output
    uₜ[:,:,1] = φ
    uₜ[:,:,2] = φₜ

    nothing
end

prob = ODEProblem(signumGordon_2d!, u₀, (0.0, tmax), (N, h, ∇²ϕ))

;

# Solution

In [ ]:
@time sol = solve(prob, RK4(); saveat=tsave, save_idxs=rsave_idxs, adaptive=false, dt=h/10)
;

# Plots

In [ ]:
#using Plots
using Interact, WebIO, PlotlyJS


### Organize data

In [ ]:
uₕₒₛₜ = similar(sol.u)
   
copyto!(uₕₒₛₜ, sol.u)

ϕ = @views uₕₒₛₜ[100]

#ϕₕₒₛₜ  = uₕₒₛₜ[1]
#∂ₜϕₕₒₛₜ = [ for _ in 1:n]

#ϕ  = uₕₒₛₜ[:,:,1]
#ϕₜ = uₕₒₛₜ[:,:,2]

;

In [ ]:
@manipulate for t in tsave
#    p1 = plot(ϕ ,st=:surface,title="\$\\phi\$")
#    p2 = plot(ϕₜ,st=:surface,title="\$\\dfrac{\\partial\\phi}{\\partial t}\$")
#    plot(p1, p2, layout=(1, 2), size=(1300, 400))
    idx = Integer(t ./ step(tsave)) + 1
    
    ϕ = @views uₕₒₛₜ[idx];
    # plot(ϕ ,st=:surface,title="\$\\phi\$")

    # Create a meshgrid for X and Y coordinates
    n = size(ϕ, 1) # assuming ϕ is a square matrix
    X = repeat(1:n, 1, n)
    Y = repeat((1:n)', n, 1)

    # Create the surface trace
    trace = surface(
        x=X, 
        y=Y, 
        z=ϕ,
    #    colorscale = "Viridis",
    )

    # Create a layout
    layout = Layout(
        title="ϕ(t=$t)",
        autosize=false,
        width=600,
        height=600 
    )

    # Create the figure and plot it
    p = plot(trace, layout)

    # If you want to save the plot as an HTML file, you can do:
    # PlotlyJS.savefig(p, "my_surface_plot.html")

end

In [ ]:
#@manipulate for t in 0:0.1:2π
#    p1 = plot(ϕ ,st=:surface,title="\$\\phi\$")
#    p2 = plot(ϕₜ,st=:surface,title="\$\\dfrac{\\partial\\phi}{\\partial t}\$")
#    plot(p1, p2, layout=(1, 2), size=(1300, 400))
#end


In [ ]:
using PlotlyJS

# Generate data for the surface plot
x = LinRange(-5, 5, 100)
y = LinRange(-5, 5, 100)
z = [exp(-(xx^2 + yy^2)/10) for xx in x, yy in y]

# Create a surface trace
surface = surface(
    x = x,
    y = y,
    z = z,
    colorscale = "Viridis",
)

# Create a layout for the plot
layout = Layout(
    title = "Surface Plot",
    scene = Scene(
        xaxis_title = "X",
        yaxis_title = "Y",
        zaxis_title = "Z"
    )
)

# Create the plot
plot(surface, layout)


In [ ]:
using PlotlyJS

# Load a heightmap from a file or generate your own data
heightmap = ϕ

# Get the dimensions of the heightmap
width, height = size(heightmap)

# Generate the x and y coordinates based on the width and height
x = range(0, stop=width, length=width)
y = range(0, stop=height, length=height)

# Create a surface trace using the heightmap data
surface = surface(
    x = x,
    y = y,
    z = heightmap,
    colorscale = "Viridis",
)

# Create a layout for the plot
layout = Layout(
    title = "Heightmap Surface Plot",
    scene = Scene(
        xaxis_title = "X",
        yaxis_title = "Y",
        zaxis_title = "Height"
    )
)

# Create the plot
plot(surface, layout)

In [ ]:
using PlotlyJS


# Create a meshgrid for X and Y coordinates
n = size(ϕ, 1) # assuming Z is a square matrix
X = repeat(1:n, 1, n)
Y = repeat((1:n)', n, 1)

# Create the surface trace
trace = surface(x=X, y=Y, z=ϕ)

# Create a layout
layout = Layout(
    title="My Surface Plot",
    autosize=false,
    width=600,
    height=600 
)

# Create the figure and plot it
p = plot(trace, layout)

# If you want to save the plot as an HTML file, you can do:
# PlotlyJS.savefig(p, "my_surface_plot.html")


In [ ]:
using PlotlyJS

n = 10

# Assume you have an n x n matrix 'Z' representing heights
Z_orig = rand(n, n) # Replace this with your actual matrix

# Create a meshgrid for X and Y coordinates
n = size(Z_orig, 1) # assuming Z is a square matrix
X = repeat(1:n, 1, n)
Y = repeat((1:n)', n, 1)

# Create a vector to hold all the frames
frames = []

# Create a vector to hold the slider steps
steps = []

# Generate the surfaces for each time step
for t in 1:10
    Z = t * Z_orig
    trace = surface(x=X, y=Y, z=Z, showscale=false)

    # Add the current surface as a frame
    push!(frames, attr(name=string(t), data=[trace]))

    # Add a step to the slider for the current time step
    push!(steps, attr(method="animate", 
                      args=[[string(t)], attr(frame=attr(duration=0, redraw=true), mode="immediate")], 
                      label=string(t)))
end

# Create a layout with the slider
layout = Layout(title="My Surface Plot", 
                width=800, 
                height=600, 
                updatemenus=[attr(type="buttons", 
                                  showactive=false, 
                                  y=0, 
                                  x=1.05, 
                                  xanchor="right", 
                                  yanchor="top", 
                                  pad=attr(t=0, r=10), 
                                  buttons=[attr(label="Play", 
                                                method="animate", 
                                                args=[nothing, attr(frame=attr(duration=500, redraw=true), fromcurrent=true, mode="immediate")])])], 
                sliders=[attr(active=0, 
                              pad=attr(t=50, b=10), 
                              steps=steps)])

# Create the initial plot
initial_trace = surface(x=X, y=Y, z=Z_orig, showscale=false)
p = plot(initial_trace, layout)

# Add the frames to the plot
#p.frames = frames

# If you want to save the plot as an HTML file, you can do:
#PlotlyJS.savefig(p, "my_surface_plot.html")


In [ ]:
using Plots
plotlyjs()  # Set the backend to PlotlyJS

n = 10

# Assume you have an n x n matrix 'Z' representing heights
Z_orig = rand(n, n) # Replace this with your actual matrix

# Create a meshgrid for X and Y coordinates
n = size(Z_orig, 1) # assuming Z is a square matrix
X = repeat(1:n, 1, n)
Y = repeat((1:n)', n, 1)

plots = Array{Any}(undef, n)

for t in 1:10    
    plots[t] = plot()
    
    p = plots[t]
    
    Z = t * Z_orig
    surface!(p, X, Y, Z, title = "My Surface Plot")
end


In [ ]:
plots[1]

In [ ]:
using Makie

n=10

t_values = range(0, stop=10, length=100)

x = repeat(1:n, 1, n)
y = repeat((1:n)', n, 1)

# define z as a function of t, x, and y
z(t, x, y) = sin.(sqrt.(x^2 + y^2) .+ t)

fig = Figure()
ax = Axis3(fig[1,1])
surface!(ax, x, y, z.(t_values[1], x, y))  # initial plot

#for t in t_values
#   surface!(ax, x, y, z(t, x, y))  # update plot
#   # You can add a pause here if needed
#   sleep(0.1)
#nd

#fig